In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1260
┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [9]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate() + LoggerMessageGate())
    
    x_prev = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1  = add!(model, constvar(:add_1, 1.0))
    x      = add!(model, randomvar(:x))
    
    x_prev_add = add!(model, make_node(+, x_prev, add_1, x))
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    y     = add!(model, datavar(:y, Float64))
    
    add_x_and_noise = add!(model, make_node(+, x, noise, y))
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [10]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [11]:
kalman(data)

From variable out of node + => Normal{Float64}(μ=1.0, σ=100.0)
From variable in1 of node + => Normal{Float64}(μ=15.056484976892479, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=15.780867624404392, σ=14.002800840280099)
From variable in1 of node + => Normal{Float64}(μ=-17.476944785710234, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=0.3166040550407153, σ=9.950371902099892)
From variable in1 of node + => Normal{Float64}(μ=14.213591173332096, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=5.9182554187133585, σ=8.137884587711596)
From variable in1 of node + => Normal{Float64}(μ=21.77953608010854, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=10.86384762304052, σ=7.053456158585984)
From variable in1 of node + => Normal{Float64}(μ=-7.955286726810719, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=8.115016079245454, σ=6.311944030978033)
From variable in1 of node + => No

10-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=14.780867624404392, σ=14.002800840280099)
 Normal{Float64}(μ=-0.6833959449592847, σ=9.950371902099892)
 Normal{Float64}(μ=4.9182554187133585, σ=8.137884587711596)
 Normal{Float64}(μ=9.86384762304052, σ=7.053456158585984)
 Normal{Float64}(μ=7.115016079245453, σ=6.311944030978033)
 Normal{Float64}(μ=9.535403600248696, σ=5.763904177042351)
 Normal{Float64}(μ=9.084785231551248, σ=5.337605126836238)
 Normal{Float64}(μ=6.5530561984390205, σ=4.993761694389224)
 Normal{Float64}(μ=7.847723710816312, σ=4.708816093480111)
 Normal{Float64}(μ=7.557871961214178, σ=4.467670516087703)